In [1]:
import tkinter as tk
import random
import numpy as np

In [2]:
class Ball:
    def __init__(self, canvas, x, y, xspeed = 0, yspeed = 0, xaccel = 0, yaccel = 0, size = 25, colour="white"):
        self.canvas = canvas
        self.col = "white"
        self.x = x
        self.y = y
        self.xspeed = xspeed
        self.yspeed = yspeed
        self.xaccel = xaccel
        self.yaccel = yaccel
        self.size = size
        self.colour = colour
        self.prot = 15
        self.index = len(balls)
        self.checkCollisions()
        self.processed = []

    def move(self):
        self.canvas.create_oval(
            self.x - self.size / 2, self.y - self.size / 2,
            self.x + self.size / 2, self.y + self.size / 2,
            fill=self.colour
        )
        self.xspeed += self.xaccel
        self.yspeed += self.yaccel
        self.x += self.xspeed
        self.y += self.yspeed
        
        self.checkCollisions()

        if self.x > self.canvas.winfo_width() - self.size / 2:
            self.xspeed = -abs(self.xspeed) #assuming collisions with walls is totally elastic
            playSynth()
        elif self.x < self.size/2:
            self.xspeed = abs(self.xspeed)
        if self.y > self.canvas.winfo_height() - self.size / 2:
            self.yspeed = -abs(self.yspeed)
            playSynth()
        elif self.y < self.size/2:
            self.yspeed = abs(self.yspeed)

    def velocity(self):
        return np.sqrt(self.xspeed**2+self.yspeed**2)

    def checkCollisions(self):        
        for i in range(len(balls)):
            if self.index != i:
                if distance(self.x, self.y, balls[i].x, balls[i].y) <= self.size / 2 + balls[i].size / 2:
                   # print("Collision detected at ", self.index, "with ball ", i)
                    writeLog(f"""Collision detected at {self.index} with ball {i}
                    OLD Ball1: {self.velocity():.2f}
                    """, 2)

                    if self.x!=balls[i].x and self.y!=balls[i].y:
                        self.xspeed, self.yspeed = calculateNewVel(self.x, self.y, self.size^2, self.xspeed, self.yspeed, balls[i].x, balls[i].y, balls[i].size^2, balls[i].xspeed, balls[i].yspeed)
                        writeLog(f"NEW Ball1: {self.velocity():.2f}\n",2)
                    
                    else:
                        self.x-=1
                        self.y-=1
                        self.xspeed, self.yspeed = calculateNewVel(self.x, self.y, self.size^2, self.xspeed, self.yspeed, balls[i].x, balls[i].y, balls[i].size^2, balls[i].xspeed, balls[i].yspeed)
                        writeLog(f"NEW Ball1: {self.velocity():.2f}\n",2)

In [3]:
def playSynth():
    # Add your playSynth logic here
    pass

def distance(x1, y1, x2, y2):
    dX = abs(x1 - x2)
    dY = abs(y1 - y2)
    return (dX ** 2 + dY ** 2) ** 0.5

def setOrigCoords(event):
    global origX, origY
    origX, origY = event.x, event.y

def addBall(ballX, ballY, ballVelX, ballVelY, ballAccelX=0, ballAccelY=0):
    global l
    balls.append(Ball(canvas, ballX, ballY, ballVelX, ballVelY, ballAccelX, ballAccelY))
    l += 1
    
def addBallEvent(event):
    newX, newY = event.x, event.y
    addBall(newX, newY, (newX-origX/10, (newY-origY)/10))
    
def dotProduct(x1,y1,x2,y2):
    return x1*x2 + y1*y2

def VSub(x1, y1, x2, y2):
    return x1-x2, y1-y2
    

def calculateNewVel(x1, y1, m1, vx1, vy1, x2, y2, m2, vx2, vy2):
    scal = (2*m2/(m1+m2))*(dotProduct(vx1-vx2, vy1-vy2, x1-x2, y1-y2)/dotProduct(x1-x2, y1-y2, x1-x2, y1-y2))
    posDifX, posDifY = VSub(x1,y1,x2,y2)
    writeLog(f"{scal}",2)
    return VSub(vx1, vy1, scal*posDifX, scal*posDifY)

In [4]:
writeFile = open("log.txt", "wt")
loggingLevel = 2 #0 = nothing, 1 = errors only, 2 = all

def writeLog(message, level=0, file = writeFile):
    if level <= loggingLevel:
        file.write(message)

writeLog("starting simulation\n", 2, writeFile)

root = tk.Tk()
root.title("Ball Bouncing Simulator")

canvas = tk.Canvas(root, width=500, height=400, bg="black")
canvas.pack()

frame = tk.Frame(root)
frame.pack()

#frame.columnconfigure(0,weight=1)
#frame.columnconfigure(1,weight=2)


ballx = tk.Entry(frame,width=10)
ballx.insert(0, "10")
ballx.grid(column=1, row=0)
#ballx.pack(padx=1,pady=1)

ballxlabel = tk.Label(frame, text="X")
#ballxlabel.pack(side=tk.LEFT)
ballxlabel.grid(column=0, row=0)


bally = tk.Entry(frame, width=10)
bally.insert(0,"10")
#bally.pack(padx=1,pady=1)
bally.grid(column=1, row=1)


ballylabel = tk.Label(frame, text="Y")
#ballylabel.pack(side=tk.LEFT)
ballylabel.grid(column=0, row=1)


ballvelx = tk.Entry(frame, width=10)
ballvelx.insert(0,"10")
#ballvelx.pack(padx=1,pady=1)
ballvelx.grid(column=1, row=2)

ballxvellabel = tk.Label(frame, text="X velocity")
#ballxvellabel.pack(side=tk.LEFT)
ballxvellabel.grid(column=0, row=2)


ballvely = tk.Entry(frame, width=10)
ballvely.insert(0,"10")
#ballvely.pack(padx=1,pady=1)
ballvely.grid(column=1, row=3)

ballyvellabel = tk.Label(frame, text="Y velocity")
#ballyvellabel.pack(side=tk.LEFT)
ballyvellabel.grid(column=0, row=3)


def addBallWithParams():
    addBall(int(ballx.get()), int(bally.get()), int(ballvelx.get()), int(ballvely.get()))
    
AddBall = tk.Button(frame, text="Submit", command = addBallWithParams)
#AddBall.pack(padx=5, pady=5)
AddBall.grid(column=1, row=4, padx=5, pady=5)

balls = []
origX, origY = 0, 0
l = 0

#canvas.bind("<Button-1>", setOrigCoords)
#canvas.bind("<ButtonRelease-1>", addBall)

def update():
    canvas.delete("all")
    for ball in balls:
        ball.move()
    root.after(30, update)
    
update()

root.mainloop()

writeFile.close()